In [ ]:
# | default_exp read_soil_file

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
# | export
from pathlib import Path
import os
import pandas as pd
from typing import Dict
from sureau_ecos_py.create_modeling_options import create_modeling_options
import numpy as np
import collections
from pandera.typing import Series
import pandera as pa

In [ ]:
# | export
# This class was created for validating the input dataframe
# If the data don't follow the structure specified the function will fail


class SoilFile(pa.SchemaModel):
    "Schema for validating the input soil parameter file"

    Name: Series[str] = pa.Field(description="Parameter names")
    Value: Series[float] = pa.Field(description="Parameter values")


def read_soil_file(
    file_path: Path,  # Path to a csv file containing parameter values i.e path/to/file_name.csv
    modeling_options: Dict = None,  # Dictionary created using the `create_modeling_options` function
    sep: str = ";",  # CSV file separator can be ',' or ';'
) -> Dict:
    "Function for reading a data frame containing information about soil characteristics"

    # Make sure that modeling_options is a dictionary ---------------------------
    assert isinstance(
        modeling_options, Dict
    ), f"modeling_options must be a dictionary not a {type(modeling_options)}"

    # Read data frame -----------------------------------------------------------
    if os.path.exists(file_path):
        # Read file
        soil_data = pd.read_csv(file_path, header=0, sep=sep)

        # Raise error if soil data don't follow the SoilFile Schema
        SoilFile.validate(soil_data)

    else:
        print(f"file: {file_path}, does not exist, check presence or spelling")

    # Setting common parameters for WB_soil (regardless of the options) ---------
    if modeling_options["pedo_transfer_formulation"] == "vg":
        # 14 params
        params = np.array(
            [
                "rfc_1",
                "rfc_2",
                "rfc_3",
                "depth_1",
                "depth_2",
                "depth_3",
                "wilting_point",
                "alpha_vg",
                "n_vg",
                "i_vg",
                "ksat_vg",
                "saturation_capacity_vg",
                "residual_capacity_vg",
                "g_soil_0",
            ],
            dtype=object,
        )

    if modeling_options["pedo_transfer_formulation"] == "campbell":
        # 12 params
        params = np.array(
            [
                "rfc_1",
                "rfc_2",
                "rfc_3",
                "depth_1",
                "depth_2",
                "depth_3",
                "wilting_point",
                "ksat_campbell",
                "saturation_capacity_campbell",
                "b_camp",
                "psie",
                "g_soil_0",
            ],
            dtype=object,
        )

    # Get only the required params ----------------------------------------------
    soil_data = soil_data[soil_data["Name"].isin(params)]

    # Make sure that no parameters are missing (12 or 14) -----------------------
    for each_parameter in params:
        # Raise error if a parameter is missing from params
        if each_parameter not in np.array(soil_data["Name"]):
            raise ValueError(
                f"{each_parameter} not provided in input soil parameter file, check presence or spelling\n"
            )

    # Make sure there are no duplicate parameters -------------------------------
    if len(soil_data["Name"]) is not len(set(soil_data["Name"])):
        raise ValueError(
            "Parameter repeated several times in input soil parameter file"
        )

    # Get values in the dataframe and get rid of the colnames. Save everything
    # in to a list for later convert it to a Dictionary
    soil_data = soil_data.values.tolist()

    return collections.defaultdict(list, dict(soil_data))

## Create modeling options dictionary

In [ ]:
modeling_options_dict = create_modeling_options(
    time_step_for_evapo=2,
    reset_swc=True,
    avoid_water_soil_transfer=True,
    constant_climate=False,
    defoliation=True,
    soil_evapo=True,
    threshold_mortality=51,
    etp_formulation="pt",
    rn_formulation="linear",
    comp_options_for_evapo="custom",
    stomatal_reg_formulation="turgor",
    transpiration_model="jarvis",
    numerical_scheme="implicit",
    pedo_transfer_formulation="vg",
)

## Read soil dataframe

In [ ]:
read_soil_file(
    "./sample_data/soil_example.csv", modeling_options=modeling_options_dict, sep=";"
)

defaultdict(list,
            {'rfc_1': 75.0,
             'rfc_2': 82.0,
             'rfc_3': 94.0,
             'wilting_point': 0.12379,
             'alpha_vg': 0.0005,
             'n_vg': 1.55,
             'i_vg': 0.5,
             'ksat_vg': 10.0,
             'saturation_capacity_vg': 1.0,
             'residual_capacity_vg': 0.098,
             'g_soil_0': 30.0,
             'depth_1': 0.2,
             'depth_2': 1.0,
             'depth_3': 4.0})